# Hands-On CUDA C/C++ 101

This is an introductory, hands-on, workshop for programming GPU devices using [CUDA](https://developer.nvidia.com/cuda-zone) C/C++. Even though we will be using a Jupyter Notebook to edit and run our code be adviced that none of the commands are actually executed here but rather on the server (be it one of the cloud instances or your local machine).

This hands-on was highly influenced by NVidia's workshops and tutorial including (but not only):

- [NVidia's qwiklab catalog](https://nvidia.qwiklab.com/catalog/)
- [CUDA by example](https://developer.nvidia.com/cuda-example)
- [CUDA C programming guide](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html)

Be sure to check them out for more in-depth information.

---

In this lab we will:

1. see a brief overview of CPU vs GPU architectures
2. discuss why GPUs are better for certain tasks often utilized in machine learning and what are the limitations
3. explain how to extend C/C++ code using CUDA so it can be executed on GPUS
4. write simple GPU enabled programs
5. mention several higher level libraries, which will make you more productive

### Jupyter Notebook

For people not acquainted with how Jupyter Notebook works, every notebook consists of a set of cells, each having a type. Each cell can be executed by navigating to it (e.g. by clicking it) and pressing **ctrl+enter** or by clicking the "Run cell" icon in the top menu.

Try and run the below cell (you can run terminal commands in a code cell by prepending them with an exclamation ! mark). The output, containing information about GPU devices running on the server, should appear below it.

In [ ]:
!nvidia-smi

You can download all the files for later use, first run the cell below:

In [ ]:
%%bash
rm -f cuda_101.zip
zip -r cuda_101.zip *

The zipped file can be downloaded from [here](/files/cuda_101.zip).

# CPU vs GPU Overview

![cpu-gpu-transistors](gpu-devotes-more-transistors-to-data-processing.png)

**CPU**
* few cores optimized for *serial processing*
* lower memory bandwith (but direct access to more memory) 
* *latency* optimized cores (faster at a single task but can only perform few at the same time)
* more instructions but slower execution

**GPU**
* hundreds/thousands of smaller, more efficient cores optimized for *multiple tasks simultaneously*
* great for compute-intensive parts of the application
* higher memory bandwidth (but direct access to less memory than CPU)
* *throughput* optimized cores (slower at single tasks but can perform more at the same time)
* limited number of instructions available but faster execution

![how-gpu-acceleration-works](how-gpu-acceleration-works.png)
*http://www.nvidia.com/object/what-is-gpu-computing.html

In a GPU accelerated appplication the compute-intensive parts are offloaded to the GPU, while the remaining of the application runs on the CPU.

## GPUs in Machine Learning

**Pros**
* ML algorithms tend to be **compute-bound** (require more computations than fetches from memory) - GPUs are intended for compute intensive tasks!
* many ML algorithms consist of highly parallelizable tasks e.g. matrix multiplications peformed over multiple iterations - GPUs are great at those

**Drawbacks**
* host RAM to GPU RAM memory copy overhead (over PCI-E)
* limited (very limited compared to host RAM) GPU RAM size
* serial tasks slow

# CUDA Overview

Here we will outline the main concepts behind the CUDA programming model (focusing on how they are exposed in C). A more in-depth explanation can be found in the [official programming guide](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html).

* **host** - CPU and RAM

* **device** - GPU and it's RAM

* **kernels** - special functions, which can be called from host code (regular C code running on the CPU) but are run on the device (GPU) N times in parallel, executed by N CUDA threads.

* **\__global__** - special CUDA C keyword used as part of a method signature to mark that method as a kernel.

* **\__host__** - methods marked with this keyword can be only called from host code and will also run on the host

* **\__device__** - methods marked with this keyword can be only called from device code and will also run on the device

As already mentioned, CUDA and GPUs are highly scalable and parallel thanks to their multithread-first programming model. Kernels are executed N times in separate threads, which are for convenience, grouped in a hierarchy of blocks and grids: 

![grid-of-thread-blocks](grid-of-thread-blocks.png)
*http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html

* **thread block** - CUDA programming model follows a well defined thread hierarchy model in which threads during execution are grouped into so called thread blocks. Thread blocks can be one, two or three dimensional. This very naturally maps to vectors, matrices and volumes.

There's a limit of threads per block since 1) all the threads in a block are expected to reside on the same processor core 2) have to share very limited resources of said core.

We can run a kernel, though, using multiple blocks each with the same number of threads. The total number of threads per kernel in such a case will be number_of_blocks * number_of_threads_per_block.

* **threadIdx.{x,y,z}** - similar to the above these variables identify the thread ID that is being executed with the current block.

* **blockIdx.{x,y,z}** - these 3 special, read-only, variables can be used within CUDA kernels to find out which block is executing the code at a given time. This, and the following read-only variables, are usually used to identify which part of data should be handled by a given kernel instance as there will be N of them running in parallel.

* **blockDim.{x,y,z}** - another special read-only variable accesible within CUDA kernels. Contrary to the previous ones these are constant and describe the total number of threads within a block.

* **grid** - similarly to threads being organized in thread blocks, thread blocks are organized into one, two or three dimensional grids. The number of blocks is dictated by the size of data and/or number of processors available.

* **methodName <<< blocks, threads_per_block>>> (parameters)** - this is how you launch a kernel. You use this syntax in your host code (regular C code which runs on the CPU) but it is executed on the device (GPU). There are numerous variants of this operator but the most basic one takes in 2 numbers: the number of blocks to run on the GPU and the second the number of threads per each block.

Running tasks on the GPU could be visualized as:

<img src="vector_add_threads.png" alt="vector_add_threads" style="width: 500px;"/>

Architecting our applications in such a way allows GPUs to take advantage of Streaming Multiprocessors (SMs) which are one of the base elements of modern GPUs (whic are built arround an array of SMs). Scheduler automatically assignes blocks to free SMs as resources become available. Thanks to this your applications will run faster on more powerful GPUs without any changes in code.

<img src="automatic-scalability.png" alt="automatic-scalability" style="width: 500px;"/>
*http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html

# Lab 0: Hello CUDA

Lets start with the most basic C program there is, a "Hello, World!". We defined one in the [lab0/lab0a.c](/edit/lab0/lab0a.c) file. You can run terminal commans from this notebook by prepending them with an exclamation mark **!**. The command below will compile the C file using **gcc**, which will output the **lab0/lab0a.out** binary and subsequently run it:

In [ ]:
!gcc lab0/lab0a.c -o lab0/lab0a.out && lab0/lab0a.out 

Now lets modify this code so it can run on the GPU. The modified version can be found here [lab0/lab0b.cu](/edit/lab0/lab0b.cu). As you can see several things changed:

1. the **hello** function has been prefixed with the **__global__** keyword making it a kernel
2. thanks to the above we can call threadIdx.x and blockIdx.x (and other variables discussed before) to get additional information on the context in which the kernel is being ran
3. the call to the *hello* method has been changed to the **<<<...>>>** syntax - in this case we will run it only using 1 block and 1 thread.
4. a call to **cudaDeviceSynchronize()** has been added as all calls to kernels are **asynchronous**, meaning our main method might return before anything got actually printed. This method is blocking and makes sure all the operations on the GPU finish before it returns.
5. The file type changed to **.cu** which is the usual suffix for files containing CUDA code (similarly **.cuh** is used for CUDA specific header files).

In [ ]:
!nvcc -o lab0/lab0b.out lab0/lab0b.cu -run

Running the above cell does a few things:

1. nvcc will compile the device code
2. C compiler will compile the host code
3. both will be linked into a single binary and ran

**Try** changing the values (modify the file, remember to **save** it and rerun the above cell) between **<<<...>>>** to:

- **<<< 2, 1 >>>**
- **<<< 1, 32 >>>** how did the output change? Is the order what you'd expect?
- **<<< 2, 16 >>>** can you set those numbers much higher?

# Lab 1: Adding arrays

Now that you know what kernels are, how to write and launch them lets try to write one which adds two arrays together storing the result in a 3rd array.

## Custom kernel

We already prepared all the boilerplate code for you in [lab1/lab1a.cu](/edit/lab1/lab1a.cu) so please just focus on the parts marked by TODO comments. You will notice certain pieces of code in the main method which we will cover in later labs, please ignore them for now.

You'll have to figure out 3 things:

- modify a regular C signature into a CUDA kernel
- use the read-only CUDA provided variables (described in the CUDA overview section) to calculate the index for each kernel instance
- in our example we will have number_of_blocks * threads_per_block instances, what if this number if bigger than the sizes of our arrays? You will need to sanity check this

After modifying the file be sure to save it either by pressing **ctrl+s** (**cmd+s**) or using the top menu File -> Save. When you're done run the cell below. Your result should show incrementing numbers from 0 to 15.

In [ ]:
!nvcc -o lab1/lab1a.out lab1/lab1a.cu -run

If you're having problems you can have a look at the solution [lab1/lab1a_solution.cu](/edit/lab1/lab1a_solution.cu).

## Thrust

Quoting the official [Thrust website](https://developer.nvidia.com/thrust):

> Thrust is a powerful library of parallel algorithms and data structures. Thrust provides a flexible, high-level interface for GPU programming that greatly enhances developer productivity. Using Thrust, C++ developers can write just a few lines of code to perform GPU-accelerated sort, scan, transform, and reduction operations orders of magnitude faster than the latest multi-core CPUs.

Thrust follows very closely the [C++ Standard Template Library](http://www.cplusplus.com/reference/stl/) API and approach exposing high level access to many useful containers and algorithms. It makes moving data between host and device memories very easy and provided algorithms are highly optimized for GPUs (taking into account things like nondeterministic order of operations and working with floating point precision numbers).

In [lab1/lab1b.cu](/edit/lab1/lab1b.cu) we use [Thrust vectors](http://docs.nvidia.com/cuda/thrust/index.html#vectors) for the same problem as before. Thrust device_vector makes it easy to allocate memory on the device and assign values. Moving data back to host memory is easily accomplished by declaring host_vector and assigning device_vector to it.

In this exercise we want to use one of the [Thrust transformation methods](https://thrust.github.io/doc/group__transformations.html). We can think about vector addition as a transformation of 2 iterators where we sum appropriate elements of each vector and put the result into the output vector.

After modifying the file you can execute the cell below to run it. You should get the same output as in the previous exercise. All necessary includes are already in the file, focus just on the TODO part.

In [ ]:
!nvcc -o lab1/lab1b.out lab1/lab1b.cu -run

[Hint1](/edit/lab1/hint1.txt)

[Hint2](/edit/lab1/hint2.txt)

If you still find it challenging you can check the [solution here](/edit/lab1/lab1b_solution.cu)

# Lab 2: Matrix multiplication

## Custom kernel

In the vector example we used single numbers when choosing the size and dimension of blocks and grids, since vectors are 1D structures this made perfect sense. Often the data we'll be working with is not 1 but 2 or 3 (or more) dimensional. In such cases it's easier to use the [dim3](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#dim3) integer vector type which can be passed to **<<< , >>>** as the number of threads per blocks and blocks per thread when launching kernels. Remember, that even though it is called dim**3** you can still use 2D (or 1D) shapes simply by setting 1 as the size of the unecessary dimensions e.g. `dim3 shape(N, M, 1)` . As you probably remember from the intro, we can access those coordinates withing grids/blocks via blockIdx, blockDim and threadIdx - all of which are also of type `dim3` and contain fields `dim3.{x|y|z}`.

One example of a task where using 2D grids and blocks would be useful is matrix multiplication. In this lab (for brevity's sake) we will implement a square 2D `A*B` matrix multiplication kernel.

There are different ways you could tackle this problem, but today we will focus on an implementation which calculates a single output cell in the output matrix C in each GPU thread. This means that each thread (kernel instance) will calculate the product of a single row of matrix A times a single column of matrix B as shown in the picture below: 

![matrix-multiplication](http://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/matrix-multiplication-without-shared-memory.png)
*http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#shared-memory

Your task is to fix all the TODOs in the [lab2/lab2_kernel.cu](/edit/lab2/lab2_kernel.cu) file:

1. fix the kernel signature
2. figure out how to obtain the row/column index for current thread using blockIdx, blockDim and threadIdx vectors
3. fix the sanity check in the kernel
4. calculate how many blocks we will need given the block dimensions

If all goes well the result of running the following cell should be "Success!".

In [ ]:
!nvcc -o lab2/lab2_kernel.out lab2/lab2_kernel.cu -run

The solution can be found [here](/edit/lab2/lab2_kernel_solution.cu).

## cuBLAS

Thrust which we used in the previous exercise is of course not the only useful CUDA library.

If in your work you use a lot of basic linear algebra subroutines (BLAS) you might be interested in the [cuBLAS](https://developer.nvidia.com/cublas) project:

> a fast GPU-accelerated implementation of the standard basic linear algebra subroutines (BLAS). Using cuBLAS APIs, you can speed up your applications by deploying compute-intensive operations to a single GPU or scale up and distribute work across multi-GPU configurations efficiently.

One such subroutine is matrix-matrix multiplication, which in cuBLAS is achieved using the [cublas< t >gemm()](http://docs.nvidia.com/cuda/cublas/index.html#cublas-lt-t-gt-gemm) function call. For example multiplying 2 **float** matrices would only take one call:

```cpp
float alpha = 1.0f;
float beta  = 0.0f;
cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, wB, hA, wA, &alpha, d_B, wB, d_A, wA, &beta, d_C, wA);
```

Where `hA, wA, hB, wB` are dimensions of A and B and `d_A, d_B, d_c` are pointers to the data on the device.

There are certain things you do need to remember while using such libraries, though, such as that CUBLAS is column primary (which might not be very natural for most programmers) or that it wasn't optimized for certain matrix shapes (for example skinny and tall ones) and you still might need to write your own kernel for such corner cases.

# Lab 3: Error handling

Debugging and handling errors is just as important, if not more important, when writing GPU code as it is when you're writing host code. Without proper error handling you will most probably not notice if something went wrong as CUDA does not really throw exceptions unless it's a faily serious offense (e.g. bad memory access).

When dealing with error handling in CUDA we need to realize first that there are 2 types of errors we need to take care of:

1. Errors during calls to blocking (**synchronous**) methods such as [cudaMalloc()](https://www.cs.cmu.edu/afs/cs/academic/class/15668-s11/www/cuda-doc/html/group__CUDART__MEMORY_gc63ffd93e344b939d6399199d8b12fef.html). Those methods can be easily recognized in the API as they always return [cudaError_t](https://www.cs.cmu.edu/afs/cs/academic/class/15668-s11/www/cuda-doc/html/group__CUDART__TYPES_gf599e5b8b829ce7db0f5216928f6ecb6.html#gf599e5b8b829ce7db0f5216928f6ecb6). Anything which is not a `cudaSuccess` is an error. We can find out more information using `cudaGetErrorString(code)`:
```cpp
cudaError_t errCode = cudaMalloc((void**) &ptr, sizeof(int));
if (errCode != cudaSuccess) { printf("Malloc error: %s\n", cudaGetErrorString(errCode)); }
```
2. Errors during kernel calls. As we remember from the first lab kernels are executed in an **asynchronous** fashion, meaning our host thread potentially can resume before our kernel actually finishes. Because of that kernels do not return any error codes. To deal with this we need to make sure our kernel is definitely finished before we try to query for potential errors. Like in the previous lab we can use [cudaDeviceSynchronize()](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d) to make the host thread wait for all the tasks in the current CUDA stream to finish. This method is a synchronous API call, which returns and error message should an exception occur during kernel execution. Therefore we can use the previous method to handle our error:
```cpp
someKernel<<<nrGrids, nrBlocks>>>();
cudaError_t errCode = cudaDeviceSynchronize();
if (errCode != cudaSuccess) { printf("Kernel error: %s\n", cudaGetErrorString(errCode)); }
```
3. If at any other point in time we need to query the CUDA context about previous errors, or we want to get synchronous errors when calling kernels, we can use either [cudaGetLastError()](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html#group__CUDART__ERROR_1g3529f94cb530a83a76613616782bd233) (which resets the context state to `cudaSuccess`) or [(cudaPeekAtLastError)](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html#group__CUDART__ERROR_1ge9f0596369c32aa43875c17c78a8fe41) (which does not reset the context):
```cpp
someKernel<<<nrGrids, nrBlocks>>>();
cudaError_t errCode = cudaGetLastError();
if (errCode != cudaSuccess) { printf("Kernel error: %s\n", cudaGetErrorString(errCode)); }
```

Now let's run [lab3/lab3.cu](/edit/lab3/lab3.cu) using the cell below, what can you tell by the output?

There are bugs in that file, your task is to use the above error handling method to figure out what's wrong and fix it so the output matches the output from lab1.

In [ ]:
!nvcc -o lab3/lab3.out lab3/lab3.cu -run

[Hint1](/edit/lab3/hint1.txt)

[Hint2](/edit/lab3/hint2.txt)

If you still find it challenging you can check the [solution here](/edit/lab3/lab3_solution.cu)

# Lab 4: Memory management

Just like in regular C/C++ memory management in CUDA is very important and often times needs to be handled manually. Contrary to host code, the GPU does not directly work with host memory (RAM) in majority of cases (unless [pinned host memory](https://devblogs.nvidia.com/how-optimize-data-transfers-cuda-cc/) is used) and instead is using several kinds of device memory. Because of this data used in your kernels has to be first transfered from the host memory via PCI-E to the device memory, which obviously causes overhead and in cases where computation gain from using GPUs is small compared to running it on the CPU the overall time might actually degrade.

A GPU has several types of memory we can use:

<img src="gpu-memory.png" alt="gpu-memory" style="width: 750px;"/>

## Device (global) memory

**Global memory** is accesible by all the threads, blocks and grids running on the **device**. This is your main memory when running GPU code, similar to RAM on the host. When moving data from the host to the device, this is where it will end up. While running a CUDA accelerated application, CUDA will also use this memory for its own use (CUDA context etc.). Just like with host memory you need to allocate it first before moving data from the host.

To use this memory you can allocate and copy data into it manually or you can take advantage of the [Unified Memory](https://devblogs.nvidia.com/unified-memory-in-cuda-6/) and managed memory.

* When doing it **manually** you will first have to allocate the memory and then copy it yourself, either synchronously or asynchronously (you can find the whole list of memory related API calls [here](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html):

`__host__ __device__ cudaError_t cudaMalloc ( void** devPtr, size_t size )` - very similarly to `malloc` on the host this function allocates a chunk of memory of size `size` on the device memory. After the call returns `devPtr` holds the pointer to the beginning of the allocated memory.

`__host__ cudaError_t cudaMemcpy ( void* dst, const void* src, size_t count, cudaMemcpyKind kind )` - very similarly to `memcpy` on the host this function copies the data between the host and device memories. The `kind` parameter specifies from where to where is the data transfered and has to be one of the following:  `cudaMemcpyHostToHost`, `cudaMemcpyHostToDevice`, `cudaMemcpyDeviceToHost`, `cudaMemcpyDeviceToDevice`, or `cudaMemcpyDefault`. Whereas the last one, `cudaMemcpyDefault` can only be selected when using a [Unified Virtual Addressing] enabled device. This call in most cases is `synchronous` in regards to the host code.

`__host__ __device__ cudaError_t cudaMemcpyAsync ( void* dst, const void* src, size_t count, cudaMemcpyKind kind, cudaStream_t stream = 0 )` - similar to `cudaMemcpy` but this call is `asynchronous` in regards to the host code, which means it might return control to the host thread before all the data is fully copied.

A simple example would look like this:

```cpp
int N = 10;
int a[N]; 
int *dev_a;

// Initialize array "a" values
a[0] = 0; a[1] = 1; ...; a[9] = 9;

// Allocate memory on the GPU
cudaMalloc( (void**)&dev_a, N * sizeof(int) ); 
    
// Copy the data from host to GPU memory
cudaMemcpy( dev_a, a, N * sizeof(int), cudaMemcpyHostToDevice );

// Run kernels
myKernel<<<1,1>>>(dev_a);
cudaDeviceSynchronize();

// Copy the data from GPU to host memory
cudaMemcpy( a, dev_a, N * sizeof(int), cudaMemcpyDeviceToHost );
```

* With the introduction of Unified Memory CUDA also introduced the so called **managed** memory. In UM/managed memory there is no distinction (for the programmer) between host and device poitners and all the data transfer is taken care of under the hood by CUDA:

`__host__ cudaError_t cudaMallocManaged ( void** devPtr, size_t size, unsigned int  flags = cudaMemAttachGlobal )` - this call allocates a chunk of memory of size `size` in the unified memory. `devPtr` is the beginning of the allocated memory. You can use this pointer both in host and device code, whereas data transfer will take place when necessary and will be handled automatically by CUDA.

In this case we don't need two arrays/pointers:

```cpp
int N = 10;
int *a;

// Allocate memory on the GPU
cudaMallocManaged( (void**)&a, N * sizeof(int) ); 

// Initialize array "a" values
a[0] = 0; a[1] = 1; ...; a[9] = 9;

// Run kernels
myKernel<<<1,1>>>(a);
cudaDeviceSynchronize();
```

* When done with allocated memory (be it manually allocated or managed memory) we do have to deallocate it always:

`__host__ __device__ ​cudaError_t cudaFree ( void* devPtr )`

Now, without checking the previous lab files, try to use first the manual memory methods and later the managed memory methods in the [lab4/lab4.cu](/edit/lab4/lab4.cu) file to get our vector addition methods to work like before:

In [ ]:
!nvcc -o lab4/lab4_manual.out lab4/lab4_manual.cu -run

In [ ]:
!nvcc -o lab4/lab4_managed.out lab4/lab4_managed.cu -run

As always you can check the solution [here for manual case](/edit/lab4/lab4_manual_solution.cu) and [here for managed case](/edit/lab4/lab4_managed_solution.cu).

### Thrust vectors

As mentioned earlier Thrust gives us convenient, high level, containers and algorithms for the GPU out of the box. Another really useful feature is simplified memory management. Two of the most fundamental Thrust containers are `thrust::host_vector<T>` and `thrust::device_vector<T>`. They mimic the STL `vector` container, former being the host equivalent and the latter being the device equivalent.

To move data to the device we can either first generate a host vector and then use the `=` operator:

```cpp
thrust::host_vector<int> h(3); 
h[0] = 0; h[1] = 1; h[2] = 2;
thrust::device_vector<int> d = h; 
```

Or we can directly initialize the device vector from host code:

```cpp
thrust::device_vector<int> d(3); 
d[0] = 0; d[1] = 1; d[2] = 2;
```

## Shared memory

Global memory on the device, as explained, is your main memory when performing tasks on the GPU. We do have also access to the so called **shared memory**, which is situated directly on the SM. Because of this it is expected to be much faster than global device memory, but can also hold much less data. This memory is accessible by all the thread in a single block and each block has its own shared memory. Whenever possible this memory should be used for performance gains.

To allocate shared memory we use the **\__shared__** keyword:

```cpp
__shared__ int a[N][N];
```

In lab2 we wrote a simple kernel for matrix multiplication, where each thread calculated a single cell of the output matrix `C`. This means that, when computing `A * B` where `A` is of size `n x m` and `B` is of times `m x k`, `A` had to be read `k` times from global device memory and `B` had to be read `n` times.

This could be sped up by putting some of the data in shared memory. As we already know every thread was computing the result for a single output cell. A block is a group of threads, which will compute a submatrix of our output matrix while using only a part of matrix `A` and a part of matrix `B` as seen on the image below. Therefore we could put those submatrices into our shared memory to minimize the number of reads from global memory.

![matrix-multiplication-with-shared-memory](http://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/matrix-multiplication-with-shared-memory.png)
*http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#shared-memory


Your task now is to modify [lab4/lab4_shared.cu](/edit/lab4/lab4_shared.cu) to use shared memory. You'll need to do several things:

1. decide on the appropriate block size
2. declare shared memory arrays holding submatrices A and B
3. fill out those submatrices with data
4. use the [__syncthreads()](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#synchronization-functions) functions before using shared memory arrays so we are sure all the threads put the data into it. `__syncthreads()` can be used inside kernels and acts as a barrier within a block of thread. During execution every thread stops where it was called and waits until all other threads within that block reach that point. It also guarantees that any access to global and shared memory done by threads from that block will be visible in the whole block

In [ ]:
!nvcc -o lab4/lab4_shared.out lab4/lab4_shared.cu -run

As always you can check the solution [here](/edit/lab4/lab4_shared_solution.cu).